In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [4]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [5]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [6]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    #print(J)
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [7]:
# pre_initial setup
import DataProvider
prov = DataProvider.DataProvider()

reading helper data on path: ./Data/Helper/backGround_street.wav
reading helper data on path: ./Data/Helper/backGround_noice.wav
reading helper data on path: ./Data/Helper/backGround_nature.wav
[new value of  16
-new value of  34
-new value of  35
----new value of  62
----new value of  65
--------new value of  72
---------new value of  77
-----------------------------------------------------------------------new value of  82
------------------------------------------------------------------------------------------------------------------------------------------------------------------------new value of  85
---------------------------------new value of  87
--------------------------------new value of  88
------------------------------------new value of  93
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
X = list()
y = list()
trainVal = list()

In [11]:
while prov.hasNext(True):
    Xt, yt = prov.next(True)
    for i in range(len(yt)):
        if yt[i] == 1:
            trainVal.append(i+1)
            
    X.append(Xt)
    y.append(yt)

In [30]:
X = np.nan_to_num(np.matrix(X))
y = np.matrix(y)

X.shape, y.shape

((4816, 1860), (4816, 5))

In [15]:
X2 = list()
y2 = list()
actualVal = list()

In [16]:
while prov.hasNext(False):
    X2t, y2t = prov.next(False)
    actualVal.append(y2t)
    
    k = np.zeros(5)
    k[y2t - 1] = 1
    
    X2.append(X2t)
    y2.append(k)

In [31]:
X2 = np.nan_to_num(np.matrix(X2))
y2 = np.matrix(y2)

X2.shape, y2.shape

((27, 1860), (27, 5))

In [56]:
# initial setup
input_size = prov.returnSizeOfEverySpectogram()
hidden_size = 2
num_labels = 5
learning_rate = 1
epochs = 50

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25
#print(params.shape)
#params = np.ones(hidden_size * (input_size + 1) + num_labels * (hidden_size + 1))

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((2, 1861), (5, 3))

In [57]:
for epoch in range(epochs):
    for index in range(len(X)):
        Xt = X[index]
        yt = y[index]
        J, grad = backprop(params, input_size, hidden_size, num_labels, Xt, yt, learning_rate)
        params -= grad
        print(J)
    print("epoch N:",epoch,"---> succeeded")

epoch N: 0 ---> succeeded
epoch N: 1 ---> succeeded
epoch N: 2 ---> succeeded
epoch N: 3 ---> succeeded
epoch N: 4 ---> succeeded
epoch N: 5 ---> succeeded
epoch N: 6 ---> succeeded
epoch N: 7 ---> succeeded
epoch N: 8 ---> succeeded
epoch N: 9 ---> succeeded
epoch N: 10 ---> succeeded
epoch N: 11 ---> succeeded
epoch N: 12 ---> succeeded
epoch N: 13 ---> succeeded
epoch N: 14 ---> succeeded
epoch N: 15 ---> succeeded
epoch N: 16 ---> succeeded
epoch N: 17 ---> succeeded
epoch N: 18 ---> succeeded
epoch N: 19 ---> succeeded
epoch N: 20 ---> succeeded
epoch N: 21 ---> succeeded
epoch N: 22 ---> succeeded
epoch N: 23 ---> succeeded
epoch N: 24 ---> succeeded
epoch N: 25 ---> succeeded
epoch N: 26 ---> succeeded
epoch N: 27 ---> succeeded
epoch N: 28 ---> succeeded
epoch N: 29 ---> succeeded
epoch N: 30 ---> succeeded
epoch N: 31 ---> succeeded
epoch N: 32 ---> succeeded
epoch N: 33 ---> succeeded
epoch N: 34 ---> succeeded
epoch N: 35 ---> succeeded
epoch N: 36 ---> succeeded
epoch N: 37

In [58]:
params

array([ 1.08211683e+01, -5.76953305e-09,  7.99580702e-09, ...,
       -4.86180274e+00, -7.61829345e-03, -7.61829343e-03])

In [59]:
testRes = []

theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

In [62]:
for index in range(len(X2)):
    X2t = X2[index]
    y2t = y2[index]
    print(X2t)
    print(y2t)
    a1, z2, a2, z3, h = forward_propagate(X2t, theta1, theta2)
    print(h)
    testRes.append((h, y2t))

[[0.10429882 0.84297186 0.22559729 ... 0.         0.         0.        ]]
[[0. 1. 0. 0. 0.]]
[[0.99005416 0.00468744 0.00992435 0.00237115 0.00756192]]
[[-1.17429471  0.07979877  1.50101852 ...  0.          0.
   0.        ]]
[[0. 0. 0. 0. 1.]]
[[0.99005416 0.00468744 0.00992435 0.00237115 0.00756192]]
[[-1.98431826 -0.13079834 -0.14884748 ...  0.          0.
   0.        ]]
[[0. 0. 0. 0. 1.]]
[[0.99005416 0.00468744 0.00992435 0.00237115 0.00756192]]
[[-0.57486361 -0.93417966 -0.88993984 ...  0.          0.
   0.        ]]
[[1. 0. 0. 0. 0.]]
[[0.99005416 0.00468744 0.00992435 0.00237115 0.00756192]]
[[-1.22655666 -0.6860289   1.25584924 ...  0.          0.
   0.        ]]
[[0. 0. 0. 0. 1.]]
[[0.99005416 0.00468744 0.00992435 0.00237115 0.00756192]]
[[-1.02249658 -0.98860562  0.24954413 ...  0.          0.
   0.        ]]
[[0. 0. 0. 1. 0.]]
[[0.99005416 0.00468744 0.00992435 0.00237115 0.00756192]]
[[-0.49393836 -0.22859827 -2.26535726 ...  0.          0.
   0.        ]]
[[0. 0. 0. 0. 

In [61]:
testRes

[(matrix([[0.99005416, 0.00468744, 0.00992435, 0.00237115, 0.00756192]]),
  matrix([[0., 1., 0., 0., 0.]])),
 (matrix([[0.99005416, 0.00468744, 0.00992435, 0.00237115, 0.00756192]]),
  matrix([[0., 0., 0., 0., 1.]])),
 (matrix([[0.99005416, 0.00468744, 0.00992435, 0.00237115, 0.00756192]]),
  matrix([[0., 0., 0., 0., 1.]])),
 (matrix([[0.99005416, 0.00468744, 0.00992435, 0.00237115, 0.00756192]]),
  matrix([[1., 0., 0., 0., 0.]])),
 (matrix([[0.99005416, 0.00468744, 0.00992435, 0.00237115, 0.00756192]]),
  matrix([[0., 0., 0., 0., 1.]])),
 (matrix([[0.99005416, 0.00468744, 0.00992435, 0.00237115, 0.00756192]]),
  matrix([[0., 0., 0., 1., 0.]])),
 (matrix([[0.99005416, 0.00468744, 0.00992435, 0.00237115, 0.00756192]]),
  matrix([[0., 0., 0., 0., 1.]])),
 (matrix([[0.99005416, 0.00468744, 0.00992435, 0.00237115, 0.00756192]]),
  matrix([[1., 0., 0., 0., 0.]])),
 (matrix([[0.99005416, 0.00468744, 0.00992435, 0.00237115, 0.00756192]]),
  matrix([[0., 1., 0., 0., 0.]])),
 (matrix([[0.990054